In [1]:
#importo las librerias necesarias

import duckdb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
#me conecto a la bbdd para extraer los datos

con = duckdb.connect("/data/diamonds_train.db",read_only=True)
con

In [3]:
#creo un dataframe por cada una de las tablas y las observo

df_d_city = con.sql("SELECT * FROM diamonds_city").df()
df_d_city[:2]

,city_id,city
0,0013c01fe0d094209b8bd3d23b8f96dbabcd01ddd2a039...,Amsterdam
1,1e73b1647343b286269d517e6f07e6e07ccef10cd7b785...,Zurich


In [4]:
df_d_clarity = con.sql("SELECT * FROM diamonds_clarity").df()
df_d_clarity[:2]

,clarity_id,clarity
0,03c358cbd92e83278fd523f58dc6a9b4b198d00728af65...,SI2
1,3f9db06236e9719b61c826b612b882fa702ec81574e44c...,VVS2


In [5]:
df_d_color = con.sql("SELECT * FROM diamonds_color").df()
df_d_color[:2]

,color_id,color
0,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,G
1,3f39d5c348e5b79d06e842c114e6cc571583bbf44e4b0e...,D


In [6]:
df_d_cut = con.sql("SELECT * FROM diamonds_cut").df()
df_d_cut[:2]

,cut_id,cut
0,388655e25e91872329272fc10128ef5354b3b19a05d7e8...,Very Good
1,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,Ideal


In [7]:
df_d_properties = con.sql("SELECT * FROM diamonds_properties").df()
df_d_properties[:2]

,index_id,cut_id,color_id,clarity_id
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,de88c121a82a06352bf1aaceba20578356408a334ba046...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,388655e25e91872329272fc10128ef5354b3b19a05d7e8...,44bd7ae60f478fae1061e11a7739f4b94d1daf917982d3...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...


In [8]:
df_d_trans = con.sql("SELECT * FROM diamonds_transactional").df()
df_d_trans[:2]

,index_id,price,city_id,carat
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,4268,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.21
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,505,89c7286890f7347ab235234e74d406596a127ae3679042...,0.32


In [9]:
df_d_dimensions = con.sql("SELECT * FROM diamonds_dimensions").df()
df_d_dimensions[:2]

,index_id,depth,table,x,y,z
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,62.4,58.0,6.83,6.79,4.25
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,63.0,57.0,4.35,4.38,2.75


In [10]:
#Ahora voy uniendo cada uno de los dataframes por las columnas comunes de cada uno de ellos
#1º uno el dataframe de transacciones con el los detalles de los diamantes por index_id

df_price = pd.merge(df_d_trans, df_d_dimensions, how='inner', on = 'index_id')
df_price[:5]

,index_id,price,city_id,carat,depth,table,x,y,z
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,4268,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.21,62.4,58.0,6.83,6.79,4.25
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,505,89c7286890f7347ab235234e74d406596a127ae3679042...,0.32,63.0,57.0,4.35,4.38,2.75
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,2686,2bd25cd960aba8b706e2b67f2bb38b750ee5384b0e9883...,0.71,65.5,55.0,5.62,5.53,3.65
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,738,89c7286890f7347ab235234e74d406596a127ae3679042...,0.41,63.8,56.0,4.68,4.72,3.00
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,4882,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.02,60.5,59.0,6.55,6.51,3.95


In [11]:
#2º añado el df de ciudad por city_id sobre el df anterior

df_city = pd.merge(df_price, df_d_city, how='inner', on = 'city_id')
df_city[:5]

,index_id,price,city_id,carat,depth,table,x,y,z,city
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,4268,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.21,62.4,58.0,6.83,6.79,4.25,Dubai
1,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,4882,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.02,60.5,59.0,6.55,6.51,3.95,Dubai
2,76a50887d8f1c2e9301755428990ad81479ee21c25b432...,14208,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,2.00,61.3,59.0,8.13,8.09,4.97,Dubai
3,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,5113,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.01,63.7,60.0,6.32,6.28,4.01,Dubai
4,3ada92f28b4ceda38562ebf047c6ff05400d4c572352a1...,2898,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,0.79,62.3,57.0,5.90,5.85,3.66,Dubai


In [12]:
#3º añado el resto de IDs que necesito para unir el resto dataframes

df_ids = pd.merge(df_city, df_d_properties, how='inner', on = 'index_id')
df_ids[:5]

,index_id,price,city_id,carat,depth,table,x,y,z,city,cut_id,color_id,clarity_id
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,4268,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.21,62.4,58.0,6.83,6.79,4.25,Dubai,de88c121a82a06352bf1aaceba20578356408a334ba046...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...
1,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,4882,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.02,60.5,59.0,6.55,6.51,3.95,Dubai,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,bd4f4a250da88f87729febc739ae97f439a14f9d38f0e3...
2,76a50887d8f1c2e9301755428990ad81479ee21c25b432...,14208,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,2.00,61.3,59.0,8.13,8.09,4.97,Dubai,de88c121a82a06352bf1aaceba20578356408a334ba046...,a83dd0ccbffe39d071cc317ddf6e97f5c6b1c87af91919...,03c358cbd92e83278fd523f58dc6a9b4b198d00728af65...
3,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,5113,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.01,63.7,60.0,6.32,6.28,4.01,Dubai,c939327ca16dcf97ca32521d8b834bf1de16573d21deda...,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,bd4f4a250da88f87729febc739ae97f439a14f9d38f0e3...
4,3ada92f28b4ceda38562ebf047c6ff05400d4c572352a1...,2898,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,0.79,62.3,57.0,5.90,5.85,3.66,Dubai,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,333e0a1e27815d0ceee55c473fe3dc93d56c63e3bee2b3...,bd4f4a250da88f87729febc739ae97f439a14f9d38f0e3...


In [13]:
#4º comienzo a añadir cada df de corte, color y claridad, comenzando por claridad

df_clar = pd.merge(df_ids, df_d_clarity, how='inner', on = 'clarity_id')
df_clar[:5]

,index_id,price,city_id,carat,depth,table,x,y,z,city,cut_id,color_id,clarity_id,clarity
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,4268,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.21,62.4,58.0,6.83,6.79,4.25,Dubai,de88c121a82a06352bf1aaceba20578356408a334ba046...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2
1,5426d2ca50f244fb43fe9eafc82da08f33f3b4f8d91408...,8870,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.21,60.1,55.0,7.01,6.91,4.18,Dubai,de88c121a82a06352bf1aaceba20578356408a334ba046...,f67ab10ad4e4c53121b6a5fe4da9c10ddee905b978d378...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2
2,e6f47e008cc58b38596e6fdf2f50a0fea93fd10543e652...,2693,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,0.70,62.5,55.0,5.73,5.70,3.57,Dubai,4e3cfaa334cbafb57a399a98fad8d3812ece460018f457...,a9f51566bd6705f7ea6ad54bb9deb449f795582d6529a0...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2
3,a05198938c6ca8cd56289c6dba6bb8aaa68dfe8e0d7a37...,14112,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.51,62.3,59.0,7.29,7.34,4.56,Dubai,388655e25e91872329272fc10128ef5354b3b19a05d7e8...,3f39d5c348e5b79d06e842c114e6cc571583bbf44e4b0e...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2
4,af5422f824076084bed9b8a09086ac59f0ed8c74eea7b1...,5962,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.21,62.3,58.0,6.82,6.79,4.24,Dubai,de88c121a82a06352bf1aaceba20578356408a334ba046...,a83dd0ccbffe39d071cc317ddf6e97f5c6b1c87af91919...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2


In [14]:
#5º continuo con color

df_color = pd.merge(df_clar, df_d_color, how='inner', on = 'color_id')
df_color[:5]

,index_id,price,city_id,carat,depth,table,x,y,z,city,cut_id,color_id,clarity_id,clarity,color
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,4268,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.21,62.4,58.0,6.83,6.79,4.25,Dubai,de88c121a82a06352bf1aaceba20578356408a334ba046...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,J
1,35041ec21b0be43fcc811118982e23e71df122c6732578...,1896,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,0.70,67.4,58.0,5.34,5.37,3.61,Dubai,f7b19afcde965ea4942b878d266f89f8ba9a5a833e60f7...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,J
2,41667f6e2629360aecaf00b20f8732e3310417ebd54b24...,3513,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.02,61.6,58.0,6.40,6.35,3.93,Dubai,de88c121a82a06352bf1aaceba20578356408a334ba046...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,J
3,7ee3819bf62f7e4563a2a9476df6e18a6cd17cceb30b92...,1287,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,0.59,61.9,58.0,5.35,5.38,3.32,Dubai,388655e25e91872329272fc10128ef5354b3b19a05d7e8...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,J
4,42e544025f96e6ee0a064873a7f2d431ca555ed0ab1f29...,6982,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.52,63.1,61.0,7.19,7.23,4.55,Dubai,c939327ca16dcf97ca32521d8b834bf1de16573d21deda...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,J


In [15]:
#6º al unir el corte ya he generado el dataframe final

df_final = pd.merge(df_color, df_d_cut, how='inner', on = 'cut_id')
df_final[:5]

,index_id,price,city_id,carat,depth,table,x,y,z,city,cut_id,color_id,clarity_id,clarity,color,cut
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,4268,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.21,62.4,58.0,6.83,6.79,4.25,Dubai,de88c121a82a06352bf1aaceba20578356408a334ba046...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,J,Premium
1,41667f6e2629360aecaf00b20f8732e3310417ebd54b24...,3513,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.02,61.6,58.0,6.40,6.35,3.93,Dubai,de88c121a82a06352bf1aaceba20578356408a334ba046...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,J,Premium
2,01f8667f50d52677bea23231a74156e4f92360d7bc3db6...,1792,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,0.77,62.3,58.0,5.86,5.80,3.63,Dubai,de88c121a82a06352bf1aaceba20578356408a334ba046...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,J,Premium
3,c3867352aab641358faec75d733af012dbe2259a014ea8...,7553,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,1.51,59.6,60.0,7.58,7.48,4.49,Dubai,de88c121a82a06352bf1aaceba20578356408a334ba046...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,J,Premium
4,0da4b104c4d8589fcb96a03aa0787549a2631935b0f499...,1176,6c425048aa7badd9d84615bd8620ca1864efd81cfdb69d...,0.57,60.2,62.0,5.40,5.33,3.23,Dubai,de88c121a82a06352bf1aaceba20578356408a334ba046...,6da43b944e494e885e69af021f93c6d9331c78aa228084...,f0b2a1d0db08cc64f85d74f1d15c2191e0e49039f4d8f2...,VS2,J,Premium


In [16]:
#reviso el dataframe final
#compruebo que no hay nulos

df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index_id    40455 non-null  object 
 1   price       40455 non-null  int64  
 2   city_id     40455 non-null  object 
 3   carat       40455 non-null  float64
 4   depth       40455 non-null  float64
 5   table       40455 non-null  float64
 6   x           40455 non-null  float64
 7   y           40455 non-null  float64
 8   z           40455 non-null  float64
 9   city        40455 non-null  object 
 10  cut_id      40455 non-null  object 
 11  color_id    40455 non-null  object 
 12  clarity_id  40455 non-null  object 
 13  clarity     40455 non-null  object 
 14  color       40455 non-null  object 
 15  cut         40455 non-null  object 
dtypes: float64(6), int64(1), object(9)
memory usage: 4.9+ MB


In [17]:
#me aseguro de que efectivamente no hay nulos

#reviso la columna precio
price = pd.isnull(df_final["price"])
print('Checking column price, if I get false there is no nulls. Result = ') 
print(price.unique())

#reviso la columna quilates
carat = pd.isnull(df_final["carat"])
print('Checking column carat, if I get false there is no nulls. Result = ') 
print(carat.unique())

#reviso la columna depth
depth = pd.isnull(df_final["depth"])
print('Checking column depth, if I get false there is no nulls. Result = ') 
print(depth.unique())

#reviso la columna table
table = pd.isnull(df_final["table"])
print('Checking column table, if I get false there is no nulls. Result = ') 
print(table.unique())

#reviso la columna x
col_x = pd.isnull(df_final["x"])
print('Checking column x, if I get false there is no nulls. Result = ') 
print(col_x.unique())

#reviso la columna y
col_y = pd.isnull(df_final["y"])
print('Checking column y, if I get false there is no nulls. Result = ') 
print(col_y.unique())

#reviso la columna z
col_z = pd.isnull(df_final["z"])
print('Checking column z, if I get false there is no nulls. Result = ') 
print(col_z.unique())

#reviso la columna city
city = pd.isnull(df_final["city"])
print('Checking column city, if I get false there is no nulls. Result = ') 
print(city.unique())

#reviso la columna clarity
clarity = pd.isnull(df_final["clarity"])
print('Checking column clarity, if I get false there is no nulls. Result = ') 
print(clarity.unique())

#reviso la columna color
color = pd.isnull(df_final["color"])
print('Checking column color, if I get false there is no nulls. Result = ') 
print(color.unique())

#reviso la columna cut
cut = pd.isnull(df_final["cut"])
print('Checking column cut, if I get false there is no nulls. Result = ') 
print(cut.unique())

#Confirmo que no hay nulos

Checking column price, if I get false there is no nulls. Result = 
[False]
Checking column carat, if I get false there is no nulls. Result = 
[False]
Checking column depth, if I get false there is no nulls. Result = 
[False]
Checking column table, if I get false there is no nulls. Result = 
[False]
Checking column x, if I get false there is no nulls. Result = 
[False]
Checking column y, if I get false there is no nulls. Result = 
[False]
Checking column z, if I get false there is no nulls. Result = 
[False]
Checking column city, if I get false there is no nulls. Result = 
[False]
Checking column clarity, if I get false there is no nulls. Result = 
[False]
Checking column color, if I get false there is no nulls. Result = 
[False]
Checking column cut, if I get false there is no nulls. Result = 
[False]


In [18]:
#limpio el df generando solo las columnas que considero relevantes para el análisis

df = df_final[["price","carat","depth","table","x","y","z","city","clarity","color","cut"]]
df

,price,carat,depth,table,x,y,z,city,clarity,color,cut
0,4268,1.21,62.4,58.0,6.83,6.79,4.25,Dubai,VS2,J,Premium
1,3513,1.02,61.6,58.0,6.40,6.35,3.93,Dubai,VS2,J,Premium
2,1792,0.77,62.3,58.0,5.86,5.80,3.63,Dubai,VS2,J,Premium
3,7553,1.51,59.6,60.0,7.58,7.48,4.49,Dubai,VS2,J,Premium
4,1176,0.57,60.2,62.0,5.40,5.33,3.23,Dubai,VS2,J,Premium
...,...,...,...,...,...,...,...,...,...,...,...
40450,1019,0.57,59.5,57.0,5.46,5.42,3.24,Surat,I1,G,Ideal
40451,6096,1.44,63.2,55.0,7.21,7.18,4.54,Luxembourg,I1,G,Ideal
40452,3459,1.02,61.2,56.0,6.47,6.50,3.97,Luxembourg,I1,G,Ideal
40453,4428,1.28,61.6,57.0,6.93,6.96,4.28,Luxembourg,I1,G,Ideal


In [19]:
#Ahora sé que para analizar las columnas numéricas, automaticamente se hace con .describe()

df.describe()

,price,carat,depth,table,x,y,z
count,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000
mean,3928.444469,0.797706,61.752841,57.446133,5.729392,5.732819,3.537154
std,3992.416147,0.475544,1.431725,2.233535,1.124453,1.146650,0.697062
min,326.000000,0.200000,43.000000,43.000000,0.000000,0.000000,0.000000
25%,945.000000,0.400000,61.000000,56.000000,4.710000,4.720000,2.910000
50%,2397.000000,0.700000,61.800000,57.000000,5.690000,5.710000,3.520000
75%,5331.000000,1.040000,62.500000,59.000000,6.540000,6.540000,4.035000
max,18823.000000,4.500000,79.000000,95.000000,10.230000,58.900000,8.060000


In [20]:
#analizo las correlaciones solo con las columnas numéricas

df_numerico = df[["price","carat","depth","table","x","y","z"]]
df_numerico.corr()

,price,carat,depth,table,x,y,z
price,1.000000,0.921935,-0.014864,0.130111,0.885848,0.866163,0.874500
carat,0.921935,1.000000,0.026528,0.183392,0.975688,0.951667,0.967570
depth,-0.014864,0.026528,1.000000,-0.293114,-0.026348,-0.030966,0.094655
table,0.130111,0.183392,-0.293114,1.000000,0.196059,0.184673,0.155189
x,0.885848,0.975688,-0.026348,0.196059,1.000000,0.973712,0.984876
y,0.866163,0.951667,-0.030966,0.184673,0.973712,1.000000,0.964828
z,0.874500,0.967570,0.094655,0.155189,0.984876,0.964828,1.000000


In [21]:
# Features + target

X = df_numerico[["carat","depth","table","x","y","z"]]
y = pd.get_dummies(df_numerico[["price"]], drop_first=True)
print(X.shape,y.shape)

(40455, 6) (40455, 1)


In [22]:
# Train + test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# Scaling - RobustScaler
scaler = RobustScaler()
scaled_data_rs = scaler.fit_transform(X_train)
scaled_data_rs
scaled_data_test = scaler.transform(X_test)
scaled_data_test

array([[ 0.484375  ,  0.66666667,  0.33333333,  0.36612022,  0.37362637,
         0.42857143],
       [ 0.484375  ,  0.73333333,  0.        ,  0.36065574,  0.39010989,
         0.4375    ],
       [-0.625     , -0.93333333,  0.66666667, -0.73224044, -0.75824176,
        -0.80357143],
       ...,
       [-0.296875  ,  1.13333333, -0.33333333, -0.32240437, -0.36263736,
        -0.26785714],
       [ 1.296875  ,  0.06666667,  0.        ,  0.90710383,  0.92857143,
         0.92857143],
       [-0.546875  , -0.13333333, -0.33333333, -0.6557377 , -0.64285714,
        -0.66071429]])

In [24]:
regressor =  LinearRegression()

In [25]:
hyperparameters = regressor.get_params()
model = regressor.fit(X_train, y_train)

In [26]:
diamond = pd.read_csv('./data/diamonds_test.csv')
diamond

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...
13480,13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam
13481,13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City
13482,13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv
13483,13483,0.70,Very Good,F,SI2,58.8,57.0,5.85,5.89,3.45,Surat


In [27]:
diamond_test = diamond[["carat","depth","table","x","y","z"]]
diamond_test

,carat,depth,table,x,y,z
0,0.79,62.7,60.0,5.82,5.89,3.67
1,1.20,61.0,57.0,6.81,6.89,4.18
2,1.57,62.2,61.0,7.38,7.32,4.57
3,0.90,63.8,54.0,6.09,6.13,3.90
4,0.50,62.9,58.0,5.05,5.09,3.19
...,...,...,...,...,...,...
13480,0.57,61.9,56.0,5.35,5.32,3.30
13481,0.71,62.2,55.0,5.71,5.73,3.56
13482,0.70,61.6,55.0,5.75,5.71,3.53
13483,0.70,58.8,57.0,5.85,5.89,3.45


In [28]:
X_test.columns == diamond_test.columns

array([ True,  True,  True,  True,  True,  True])

In [29]:
y_pred = model.predict(X_test)

print('Model:', regressor, '\n')
print('Model hyperparameters:', hyperparameters, '\n')
#print('Ground truth target:', y_test, '\n')
print('Predicted target:', y_pred, '\n')

Model: LinearRegression() 

Model hyperparameters: {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False} 

Predicted target: [[5139.12338674]
 [5230.4764621 ]
 [ 444.25088186]
 ...
 [1386.89520583]
 [9776.83759535]
 [ 845.79282026]] 



In [30]:
y_pred.shape

(8091, 1)

In [31]:
type(y_pred)

numpy.ndarray

In [32]:
r2_ = model.score(X_test, y_test)
r2_

0.8516075394641318

In [38]:
v = pd.DataFrame(y_pred)
v.columns
type(v)
len(v)

8091

In [34]:
y_pred

array([[5139.12338674],
       [5230.4764621 ],
       [ 444.25088186],
       ...,
       [1386.89520583],
       [9776.83759535],
       [ 845.79282026]])

In [39]:
len(diamond_test)

13485

In [40]:
y_hat = model.predict(diamond_test)

In [41]:
y_hat.shape

(13485, 1)

In [42]:
y_hat

array([[3277.99002433],
       [7103.52649606],
       [9705.48840468],
       ...,
       [3123.61271949],
       [3406.31967003],
       [1045.21966138]])

In [47]:
type(y_hat)

numpy.ndarray

In [95]:
df_pred = pd.DataFrame(data=y_hat)
df_pred

,0
0,3277.990024
1,7103.526496
2,9705.488405
3,4476.510912
4,1278.894369
...,...
13480,2064.778015
13481,3155.041801
13482,3123.612719
13483,3406.319670


In [94]:
df_pred = df_pred.reset_index(drop=True)
df_pred

AttributeError: 'NoneType' object has no attribute 'reset_index'

In [93]:
type(df_pred)

NoneType

In [88]:

df_rename = df_pred.rename(columns={"index": "id", "o": "price"})
df_rename

AttributeError: 'NoneType' object has no attribute 'rename'

In [82]:
final_prediction = df_rename.rename(columns={df_rename.columns[1]: "price" })
final_prediction

,id,price
0,0,3277.990024
1,1,7103.526496
2,2,9705.488405
3,3,4476.510912
4,4,1278.894369
...,...,...
13480,13480,2064.778015
13481,13481,3155.041801
13482,13482,3123.612719
13483,13483,3406.319670


In [97]:
export = final_prediction.to_csv('./data/first_test.csv', index=False)

In [98]:
pd.read_csv('./data/first_test.csv').head()

,id,price
0,0,3277.990024
1,1,7103.526496
2,2,9705.488405
3,3,4476.510912
4,4,1278.894369
